In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import seaborn as sns
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score
import matplotlib.pyplot as plt

In [2]:
yelp_train_businesses = pd.read_csv("yelp_csv_dataset/yelp_train_businesses.csv")

In [4]:
yelp_train_reviews = pd.read_csv("yelp_csv_dataset/yelp_train_reviews.csv")

In [6]:
yelp_train_users = pd.read_csv("yelp_csv_dataset/yelp_train_users.csv")

In [7]:
yelp_train_users
# replace elite with 0s

,Unnamed: 0,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,1737812,M9JLw9h8ll19vtiQuPhrlA,Jiana,5,2020-10-15 21:18:43,2,0,0,NaN,None,...,0,0,0,0,0,0,0,0,0,0
1,879220,eqObo19FcFYkibgmwQdChw,Brooke,9,2019-06-23 20:58:13,2,0,0,NaN,"mxvAHMysY7JXnz4ycfdGGw, 3XRoPBbMi01Hgmz57FRTWA...",...,0,0,0,0,0,0,0,0,0,0
2,588190,aHV8PMDudit3utJTrEVPwA,j,92,2011-05-10 19:45:11,44,4,11,NaN,None,...,0,0,0,0,1,1,0,0,0,0
3,1099996,tVSOnGuWOaufSaG9QaSH5g,Bethany,4,2017-07-08 16:36:11,2,0,1,NaN,eum-72q-LLnh0FmfBPaVuQ,...,0,0,0,0,0,0,0,0,0,0
4,1293289,L_dBsu2zFmc_dD7U_qz2RA,Thomas,7,2018-02-27 05:41:48,0,0,0,NaN,"FhrWmg-YHxTuLNl79MUXkw, gB7buT5SEk9BDtsAEiCBFw",...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1490917,491263,fAsH64iGFjvtg-nHMEGWgA,Brook,28,2014-12-26 02:36:39,9,0,5,NaN,"IUeQNUlzNLAevue-uBYJJw, Fsn9qr7_6Fa1V3B4HCEnyg...",...,0,0,0,0,0,0,0,0,0,0
1490918,1840200,FYBD1Es41n2oqZ0LVKscjQ,Jennifer,1,2018-09-01 18:19:00,0,0,0,NaN,"mVlQrYnxDRjTn9Fw_RXJFQ, b5C8cPxf1D9Q0PYkU7ahCw...",...,0,0,0,0,0,0,0,0,0,0
1490919,470924,537ROE1N4h3HF7Ew788WDA,Greta,2,2015-11-18 17:34:11,0,0,0,NaN,"mWL4fnvNw9q8mcLb9YL4pg, bxRL5tN1vT3Rvsdf_UcUzQ...",...,0,0,0,0,0,0,0,0,0,0
1490920,491755,O5F3c2lw7nT1UDIAHIuwpw,Erin,2,2014-11-30 06:15:44,0,0,0,NaN,"vWW8kXLkRw0BITmQF3DzYw, 4iGpIIShFLqvw7kwzDwJtw...",...,0,0,0,0,0,0,0,0,0,0


In [9]:
yelp_train_checkins = pd.read_csv("yelp_csv_dataset/yelp_train_checkins.csv")

In [10]:
yelp_train_checkins

,Unnamed: 0,business_id,date
0,115105,rx1VeHzhyLdMSjdHqU5f2A,"2010-02-02 23:38:19, 2010-07-25 15:39:41, 2011..."
1,128242,yDI9hTd-Xi8GCKddXXThhg,"2010-07-17 17:22:41, 2010-07-21 15:53:34, 2010..."
2,123639,w2iLl2wtYqn0M33cer7OUA,"2014-03-04 00:08:25, 2014-03-06 00:25:09, 2014..."
3,74244,Z17ByK-SmAZn4MQG07l5TA,"2014-09-11 14:09:34, 2015-03-11 21:42:35, 2015..."
4,7615,2c2n-VxWSyM6Jx6gCmBSZw,"2010-12-27 19:54:51, 2011-04-21 00:06:18, 2013..."
...,...,...,...
98942,73349,YawgmpPF0PL5_fzU6U3jhg,"2013-05-11 02:57:59, 2016-04-03 03:46:50, 2016..."
98943,109259,p7tSGmWpFV246aYhwODgBg,"2016-04-25 22:38:26, 2016-06-11 22:57:27, 2016..."
98944,50057,NLIcXAQZn6bBSnE-f-khQw,2017-04-30 17:47:19
98945,5192,1R83CFvhCxnKymRJWYmSvQ,"2016-10-01 15:39:27, 2017-09-30 13:41:52, 2019..."


In [5]:
yelp_train_reviews

,Unnamed: 0,user_id,business_id,text,date,compliment_count
0,619518,-QUL8TEm1CuwqO_TQMDrEw,9iDHCh_a3q9KStftkzHlZA,"Amazing service, excellent and very juicy burg...",2015-05-06 23:42:14,0
1,512883,jm2YCr3RulMkUFcLUQ9m9w,yaqoUczq_zbjXX7RCZF6Zg,E,2019-11-13 18:47:32,0
2,24633,CCDtU0QIVBzJhh0b7ni7aA,fC03dk7N8XV-RzGh0UJgmA,Good ice cream but terrible portions,2017-07-30 19:32:13,0
3,228575,TTa7U5Zexn3uHTb3INS5Ng,0cddB9WRKcQhzAEXEav81w,Mason jars full of vanilla bean sugar! The per...,2013-12-17 18:50:32,0
4,164815,IJ0kPn8RC1Ce0mWD0SmFng,idP674ti6a8yg8z2xFcCgA,"Great location. Attentive, helpful service. We...",2017-12-12 23:42:21,0
...,...,...,...,...,...,...
681681,491263,ZFMCV7aa-QKd2338kmNZQw,MA0dGvzwvP7APePRJp8qyg,WIFI PASSWORD - boisecoop,2018-08-08 22:31:06,0
681682,791624,bZNDO-iTy6DO49EY0u6_Mg,2aZNVfzU-3672SjRHnPoOA,"Not worth the price, especially this kid size ...",2019-08-21 18:35:55,0
681683,470924,5cLNAh7TLgRz3-sktrlDWw,Ff5GKVSfERri4wStc6kNbw,"The worst service ever! Slow at a snails pace,...",2013-11-05 21:57:46,0
681684,491755,9sWguKNI1QxPxD9-u97vLg,_LYVAMVWnR-v1_hnLhhJZw,Jesus has alleviated pain I have been dealing ...,2017-01-16 03:30:05,0


In [13]:
yelp_train_businesses['attributes']


# Figuring out most popular attributes

# Price Range, Alcohol, Ambience, Romantic

# Multi-Class Classiication Problem

0        {'Alcohol': "'none'", 'GoodForKids': 'True', '...
1        {'Alcohol': "'none'", 'RestaurantsGoodForGroup...
2        {'WiFi': "u'free'", 'BusinessAcceptsCreditCard...
3        {'Caters': 'False', 'BusinessAcceptsCreditCard...
4                   {'BusinessAcceptsCreditCards': 'True'}
                               ...                        
88208    {'ByAppointmentOnly': 'False', 'RestaurantsPri...
88209    {'RestaurantsGoodForGroups': 'False', 'Busines...
88210    {'NoiseLevel': "u'quiet'", 'RestaurantsDeliver...
88211    {'RestaurantsPriceRange2': '1', 'OutdoorSeatin...
88212    {'RestaurantsTakeOut': 'True', 'RestaurantsTab...
Name: attributes, Length: 88213, dtype: object

In [19]:
# focus on a state, best place to open up cuisine at this state
    # target - city, latitude, longitude (overall location in the state)
    # features - businessAttributes, categories, hours, reviews

yelp_train_businesses

,Unnamed: 0,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,112207,LAtPOWQ6fUvaU64Yp9hyIg,Mama's Soul Food Restaurant,3701 E Dr M L K Jr Blvd,Tampa,FL,33610,27.981229,-82.418096,4.0,114,1,"{'Alcohol': ""'none'"", 'GoodForKids': 'True', '...","Restaurants, Soul Food, Southern, Seafood","{'Wednesday': '11:0-20:0', 'Thursday': '11:0-2..."
1,53084,NbOWECn3ilz4gWL6dm5P6g,Frieda,320 Walnut St,Philadelphia,PA,19106,39.947101,-75.147183,4.5,193,1,"{'Alcohol': ""'none'"", 'RestaurantsGoodForGroup...","Cafes, Restaurants, Breakfast & Brunch, Coffee...","{'Monday': '8:0-15:0', 'Tuesday': '8:0-15:0', ..."
2,73524,pg8XTtztUXc-PxRYUVoAvw,The Hotel Zamora,3701 Gulf Blvd,St. Pete Beach,FL,33706,27.713450,-82.737262,3.5,227,1,"{'WiFi': ""u'free'"", 'BusinessAcceptsCreditCard...","Bars, Cocktail Bars, Restaurants, Hotels & Tra...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
3,147283,GboEx9CeJTneBmrM43xr4Q,Legacy Wine & Spirits,720 Legacy Ct,Smyrna,TN,37167,35.952426,-86.517312,4.5,11,1,"{'Caters': 'False', 'BusinessAcceptsCreditCard...","Beer, Wine & Spirits, Food","{'Monday': '8:0-23:0', 'Tuesday': '8:0-23:0', ..."
4,148797,-OxEfm9dsum2ooTUyR2uHA,Sierra Express Hand Car Wash & Express Detailing,2331 Kietzke Ln,Reno,NV,89502,39.501143,-119.789620,2.0,146,1,{'BusinessAcceptsCreditCards': 'True'},"Automotive, Car Wash, Auto Detailing","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88208,63829,dEMzer_93gRNNV8c-ByiSA,G's Barber Shop,1722 E Speedway Blvd,Tucson,AZ,85719,32.235687,-110.946139,4.0,17,1,"{'ByAppointmentOnly': 'False', 'RestaurantsPri...","Barbers, Beauty & Spas","{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ..."
88209,125416,Mc-YbGuvj3qYlkMfGjw6dQ,St. Louis Fish & Chicken Grill,200 N Florissant Rd,Ferguson,MO,63135,38.748729,-90.306995,4.0,16,1,"{'RestaurantsGoodForGroups': 'False', 'Busines...","Seafood, American (Traditional), Restaurants","{'Monday': '10:0-23:0', 'Tuesday': '10:0-23:0'..."
88210,6580,ksma5FPvWkaZvcwHz_bZPA,McDonald's,2130 W River Rd,Tucson,AZ,85704,32.305722,-111.012812,1.5,25,1,"{'NoiseLevel': ""u'quiet'"", 'RestaurantsDeliver...","Restaurants, Fast Food, Burgers, Coffee & Tea,...","{'Monday': '4:30-0:0', 'Tuesday': '4:30-0:0', ..."
88211,99254,2gW2AflncKzHvmSvel2LBw,McDonald's,2601 Thonotosassa Rd,Plant City,FL,33563,28.025141,-82.150652,1.5,53,1,"{'RestaurantsPriceRange2': '1', 'OutdoorSeatin...","Restaurants, Coffee & Tea, Food, Burgers, Fast...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."


In [23]:
yelp_train_businesses.groupby(['state'])['state'].agg('count')

state
AB     3096
AZ     5801
CA     2972
DE     1345
FL    15800
HI        1
ID     2711
IL     1243
IN     6691
LA     5688
MO     6695
MT        1
NC        1
NJ     4956
NV     4344
PA    19686
SD        1
TN     7178
TX        2
WA        1
Name: state, dtype: int64

In [24]:
yelp_test_businesses = pd.read_csv('yelp_csv_dataset/yelp_test_businesses.csv')

In [26]:
yelp_test_businesses.groupby(['state'])['state'].agg('count')

state
AB     818
AZ    1572
CA     799
CO       1
DE     331
FL    4149
ID     705
IL     324
IN    1739
LA    1580
MI       1
MO    1741
NJ    1328
NV    1186
PA    5297
TN    1951
VI       1
VT       1
Name: state, dtype: int64

In [ ]:
# Part 1 - Predict business attributes by using Review and Tip textual information

''' 
You need to create your own training and testing sets
(e.g., for business has more than 50 reviews + tips and
more than 3 attributes); you may need to focus on
popular attributes
'''
# Part 2 - Our own task

In [ ]:
# Classification or Regression Problem? 

# Question we're trying to answer

# Feature Selection / Importance

# Preprocessing

# Models - NLTK, RF, DT

# Hyperparameter Tuning

# Error Histograms?

# Feature Importance/Selection

# Visualizations

# Presentation